<a href="https://colab.research.google.com/github/kghilothia06/music-generation/blob/master/Model_building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from music21 import converter, stream, chord ,note,instrument
import numpy as np
import pickle
import glob

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Reading a MIDI file

In [3]:
#converting a file into stream.score object
song1 = converter.parse("/content/drive/My Drive/midi_songs/AT.mid")
song1

<music21.stream.Score 0x7f850e8bff28>

In [4]:
song1.show('midi')

In [5]:
song1.show('text')

{0.0} <music21.stream.Part 0x7f850e678f28>
    {0.0} <music21.instrument.Piano Piano>
    {0.0} <music21.instrument.Instrument >
    {0.0} <music21.instrument.Instrument >
    {0.0} <music21.instrument.AcousticBass Acoustic Bass>
    {0.0} <music21.instrument.Instrument >
    {0.0} <music21.instrument.Vocalist Voice>
    {0.0} <music21.instrument.Instrument >
    {0.0} <music21.instrument.Instrument >
    {0.0} <music21.instrument.Piano Piano>
    {0.0} <music21.instrument.ElectricOrgan Electric Organ>
    {0.0} <music21.instrument.Vocalist Voice>
    {0.0} <music21.instrument.Guitar Guitar>
    {0.0} <music21.instrument.Instrument >
    {0.0} <music21.instrument.Guitar Guitar>
    {0.0} <music21.tempo.MetronomeMark animato Quarter=120.0>
    {0.0} <music21.tempo.MetronomeMark allegro moderato Quarter=130.0>
    {0.0} <music21.key.Key of g# minor>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.stream.Voice 0x7f850e8d54a8>
        {0.0} <music21.note.Rest rest>
        {

In [6]:
#flattening this 'text' into a single list of notes and chords
element_to_parse = song1.flat.notes
len(element_to_parse)

1843

In [7]:
#traversing the falttened list - it will be collection of notes and chords
for e in element_to_parse:
    print(e,e.offset)

<music21.chord.Chord B-2 E-3 E-4 B-3> 4.0
<music21.note.Note E-> 4.0
<music21.chord.Chord F#2 G#5> 4.0
<music21.chord.Chord B-3 E-4> 4.25
<music21.chord.Chord B-3 E-4> 4.5
<music21.chord.Chord F#2 G#5> 5.0
<music21.chord.Chord E-3 E-4 B-3 B-2> 6.0
<music21.note.Note B-> 6.0
<music21.chord.Chord G#5 F#2> 6.0
<music21.chord.Chord E-4 B-3> 6.25
<music21.chord.Chord E-4 B-3> 6.5
<music21.note.Note B-> 7.0
<music21.chord.Chord B-3 B-2 E-4> 7.5
<music21.note.Note B-> 7.75
<music21.note.Note E-> 8.0
<music21.chord.Chord E-4 B-3> 8.0
<music21.note.Note E-> 8.0
<music21.chord.Chord F#2 G#5> 8.0
<music21.note.Note E-> 8.25
<music21.chord.Chord B-2 B-3 E-3> 8.5
<music21.note.Note B-> 8.75
<music21.chord.Chord E-3 E-4> 9.0
<music21.note.Note B-> 9.0
<music21.chord.Chord G#5 F#2> 9.0
<music21.note.Note E-> 9.25
<music21.chord.Chord G#4 G#3> 9.5
<music21.chord.Chord E-3 E-4> 9.5
<music21.note.Note E-> 9.5
<music21.note.Note G#> 9.75
<music21.note.Note E-> 9.75
<music21.chord.Chord B2 B3 G#4> 10.0
<m

In [0]:
#preparing a training example from 'song1'
notes = []
for e in element_to_parse:
    #if e is a note, store its pitch
    if(isinstance(e,note.Note)):
        notes.append(str(e.pitch))
    #if e is a chord, store its each constituent note with '+'
    elif(isinstance(e,chord.Chord)):
        notes.append("+".join(str(n) for n in e.normalOrder))
   

In [9]:
notes

['10+3',
 'E-3',
 '6+8',
 '10+3',
 '10+3',
 '6+8',
 '10+3',
 'B-3',
 '6+8',
 '10+3',
 '10+3',
 'B-2',
 '10+3',
 'B-3',
 'E-3',
 '10+3',
 'E-3',
 '6+8',
 'E-3',
 '10+3',
 'B-3',
 '3',
 'B-3',
 '6+8',
 'E-4',
 '8',
 '3',
 'E-4',
 'G#4',
 'E-4',
 '8+11',
 'E-4',
 'B-3',
 '6+8',
 'B3',
 '6',
 '3',
 'B3',
 'F#4',
 'E-4',
 'F#4',
 'E-4',
 '6+8',
 '10+3',
 'E-3',
 '6+8',
 '10+3',
 '10+3',
 '6+8',
 '10+3',
 'B-3',
 '6+8',
 '10+3',
 '10+3',
 'B-2',
 '10+3',
 'E-3',
 '10+3',
 'E-3',
 '10+3',
 'E-3',
 '6+8',
 'F#3',
 '6+10',
 'B-3',
 '3',
 'B-3',
 '6+8',
 'E-4',
 '3',
 'C#5',
 'E-4',
 'C#5',
 'E-4',
 'B3',
 '11+1',
 'E-4',
 'B-3',
 '6+8',
 'B3',
 '3',
 'B4',
 'B3',
 '11+3',
 '11+3',
 'B-2',
 '10',
 '3',
 'E-3',
 '4+6+8',
 '10+3',
 'E-3',
 '6+8',
 'C#3',
 '10+3',
 'E-3',
 'B-3',
 '4+6+8',
 '10+3',
 '10+3',
 'E-3',
 'B-2',
 '10+3',
 'C#3',
 '10+3',
 'E-3',
 'B-3',
 'E-3',
 '6+8',
 'E-3',
 '10+3',
 'B-3',
 '3',
 'B-3',
 '6+8',
 'E-4',
 '3',
 '8',
 '3+6',
 'E-4',
 'G#4',
 '8+11',
 'E-4',
 'G#3',
 'B-

In [10]:
len(notes)

1843

# Similarly, preprocessing all the midi files

In [11]:
notes_list = []
for file in glob.glob("/content/drive/My Drive/midi_songs/*.mid"):
    song = converter.parse(file)
    print("parsing" + file)
    element_to_parse = song.flat.notes
    for e in element_to_parse:
        #if e is a note, store its pitch
        if(isinstance(e,note.Note)):
            notes_list.append(str(e.pitch))
        #if e is a chord, store its each constituent note with '+'
        elif(isinstance(e,chord.Chord)):
            notes_list.append("+".join(str(n) for n in e.normalOrder))

parsing/content/drive/My Drive/midi_songs/8.mid
parsing/content/drive/My Drive/midi_songs/0fithos.mid
parsing/content/drive/My Drive/midi_songs/AT.mid
parsing/content/drive/My Drive/midi_songs/BlueStone_LastDungeon.mid
parsing/content/drive/My Drive/midi_songs/Cids.mid
parsing/content/drive/My Drive/midi_songs/Eternal_Harvest.mid
parsing/content/drive/My Drive/midi_songs/DOS.mid
parsing/content/drive/My Drive/midi_songs/EyesOnMePiano.mid
parsing/content/drive/My Drive/midi_songs/FF3_Third_Phase_Final_(Piano).mid
parsing/content/drive/My Drive/midi_songs/FF3_Battle_(Piano).mid
parsing/content/drive/My Drive/midi_songs/FF4.mid
parsing/content/drive/My Drive/midi_songs/FF6epitaph_piano.mid
parsing/content/drive/My Drive/midi_songs/FF8_Shuffle_or_boogie_pc.mid
parsing/content/drive/My Drive/midi_songs/FFIII_Edgar_And_Sabin_Piano.mid
parsing/content/drive/My Drive/midi_songs/FFIXQuMarshP.mid
parsing/content/drive/My Drive/midi_songs/FFIX_Piano.mid
parsing/content/drive/My Drive/midi_songs/F

In [12]:
#total no. of notes
len(notes_list)

60498

In [13]:
#no.of unique notes/classes
n_vocab = len(set(notes_list))
n_vocab

359

In [14]:
notes_list

['C5',
 'E4',
 'C2',
 'C5',
 'E4',
 'C2',
 'G4',
 'G4',
 'C5',
 'C5',
 'D5',
 '10+2',
 'B-1',
 'D5',
 '10+2',
 'B-1',
 'F4',
 'F4',
 'B-4',
 'B-4',
 'F5',
 '9+0',
 'F1',
 'F5',
 '9+0',
 'F1',
 'A5',
 '9+0',
 'F1',
 'A5',
 '9+0',
 'F1',
 'G5',
 '11+2',
 'G1',
 'G5',
 '11+2',
 'G1',
 'E5',
 'C5',
 'C2',
 'E5',
 'C5',
 'C2',
 'C2',
 'C2',
 'G4',
 'G4',
 'C5',
 'C5',
 'D5',
 'G1',
 'D5',
 'G1',
 'G4',
 'B-4',
 'G4',
 'B-4',
 'G4',
 'B-4',
 'G4',
 'B-4',
 'G1',
 'G1',
 'G4',
 'C5',
 'C2',
 'G4',
 'C5',
 'C2',
 'C2',
 'C2',
 'A3',
 'G4',
 'A3',
 'G4',
 'C4',
 'C5',
 'C4',
 'C5',
 'B-3',
 'B-4',
 'G1',
 'B-3',
 'B-4',
 'G1',
 'D4',
 'B-4',
 'D4',
 'B-4',
 'C4',
 'C5',
 'C4',
 'C5',
 'F4',
 'D5',
 'F4',
 'D5',
 'G1',
 'G1',
 'G4',
 'E5',
 'C2',
 'G4',
 'E5',
 'C2',
 'C2',
 'C2',
 'G4',
 'D5',
 'G4',
 'D5',
 'A4',
 'E5',
 'A4',
 'E5',
 'A4',
 'F5',
 'G1',
 'A4',
 'F5',
 'G1',
 'F4',
 'D5',
 'F4',
 'D5',
 'F4',
 'D5',
 'F4',
 'D5',
 'B-4',
 'B-5',
 'G1',
 'B-4',
 'B-5',
 'G1',
 'B-4',
 'F5',
 'B

# Preparing Sequential Data for LSTM

In [15]:
pitch_names = sorted(set(notes_list))
len(pitch_names)

359

In [0]:
#preparing a mapping between elements of pitch_names and int
ele_to_int = dict((ele,num) for num , ele in enumerate(pitch_names))

In [17]:
ele_to_int

{'0': 0,
 '0+1': 1,
 '0+1+5': 2,
 '0+1+6': 3,
 '0+2': 4,
 '0+2+3+7': 5,
 '0+2+4+7': 6,
 '0+2+5': 7,
 '0+2+6': 8,
 '0+2+7': 9,
 '0+3': 10,
 '0+3+5': 11,
 '0+3+5+8': 12,
 '0+3+6': 13,
 '0+3+6+8': 14,
 '0+3+6+9': 15,
 '0+3+7': 16,
 '0+4': 17,
 '0+4+5': 18,
 '0+4+6': 19,
 '0+4+7': 20,
 '0+5': 21,
 '0+5+6': 22,
 '0+6': 23,
 '1': 24,
 '1+2': 25,
 '1+2+4+6+8+10': 26,
 '1+2+6': 27,
 '1+2+6+8': 28,
 '1+3': 29,
 '1+3+5': 30,
 '1+3+5+8': 31,
 '1+3+6': 32,
 '1+3+7': 33,
 '1+3+8': 34,
 '1+4': 35,
 '1+4+6': 36,
 '1+4+6+9': 37,
 '1+4+7': 38,
 '1+4+7+10': 39,
 '1+4+7+9': 40,
 '1+4+8': 41,
 '1+5': 42,
 '1+5+8': 43,
 '1+5+9': 44,
 '1+6': 45,
 '1+7': 46,
 '10': 47,
 '10+0': 48,
 '10+0+2+5': 49,
 '10+0+3': 50,
 '10+0+4': 51,
 '10+0+5': 52,
 '10+1': 53,
 '10+1+3': 54,
 '10+1+3+5+6': 55,
 '10+1+3+6': 56,
 '10+1+4': 57,
 '10+1+4+6': 58,
 '10+1+5': 59,
 '10+11': 60,
 '10+11+3': 61,
 '10+11+3+5': 62,
 '10+2': 63,
 '10+2+3': 64,
 '10+2+4': 65,
 '10+2+5': 66,
 '10+3': 67,
 '11': 68,
 '11+0': 69,
 '11+0+4': 70,
 

Now , we prepare X and y: 
We take first 100 nodes and y the 101th node,
then shift by 1 , take x as 1 to 101th node and y as 102th node and so on..


In [0]:
seq_length = 100
network_input = []
network_output = []
for i in range(0, len(notes_list)-seq_length):
    seq_in = notes_list[i : i+seq_length]
    seq_op = notes_list[i+seq_length]
    
    #convert seq_in and seq_op to int
    network_input.append([ele_to_int[ch] for ch in seq_in])
    network_output.append(ele_to_int[seq_op])    

In [19]:
n_samples = len(network_input)
n_samples

60398

In [20]:
network_input = np.reshape(network_input , (n_samples,seq_length,1))
network_input.shape

(60398, 100, 1)

In [21]:
network_output

[356,
 333,
 310,
 356,
 333,
 310,
 310,
 310,
 356,
 320,
 356,
 320,
 286,
 333,
 286,
 333,
 286,
 345,
 353,
 286,
 345,
 353,
 344,
 320,
 344,
 320,
 344,
 320,
 344,
 320,
 292,
 293,
 353,
 292,
 293,
 353,
 292,
 345,
 292,
 345,
 292,
 345,
 292,
 345,
 286,
 333,
 310,
 286,
 333,
 310,
 310,
 310,
 356,
 320,
 356,
 320,
 286,
 333,
 286,
 333,
 292,
 345,
 353,
 292,
 345,
 353,
 356,
 320,
 356,
 320,
 292,
 345,
 292,
 345,
 320,
 293,
 320,
 293,
 353,
 353,
 313,
 356,
 310,
 313,
 356,
 310,
 357,
 313,
 357,
 313,
 310,
 310,
 313,
 356,
 313,
 356,
 292,
 353,
 292,
 353,
 286,
 286,
 320,
 320,
 356,
 353,
 356,
 353,
 292,
 344,
 292,
 344,
 313,
 332,
 310,
 313,
 332,
 310,
 332,
 332,
 344,
 344,
 356,
 356,
 357,
 357,
 310,
 310,
 356,
 356,
 313,
 313,
 356,
 356,
 292,
 353,
 292,
 353,
 286,
 286,
 320,
 320,
 356,
 353,
 356,
 353,
 292,
 332,
 292,
 332,
 344,
 344,
 313,
 356,
 310,
 313,
 356,
 310,
 357,
 313,
 357,
 313,
 310,
 310,
 313,
 356,
 313

In [22]:
normalised_network_input = network_input/float(n_vocab)
normalised_network_input 

array([[[0.8718663 ],
        [0.92479109],
        [0.86350975],
        ...,
        [0.8913649 ],
        [0.98328691],
        [0.98328691]],

       [[0.92479109],
        [0.86350975],
        [0.8718663 ],
        ...,
        [0.98328691],
        [0.98328691],
        [0.99164345]],

       [[0.86350975],
        [0.8718663 ],
        [0.92479109],
        ...,
        [0.98328691],
        [0.99164345],
        [0.9275766 ]],

       ...,

       [[0.86908078],
        [0.44011142],
        [0.81058496],
        ...,
        [0.39832869],
        [0.74651811],
        [0.74651811]],

       [[0.44011142],
        [0.81058496],
        [0.28969359],
        ...,
        [0.74651811],
        [0.74651811],
        [0.74373259]],

       [[0.81058496],
        [0.28969359],
        [0.8913649 ],
        ...,
        [0.74651811],
        [0.74373259],
        [0.74651811]]])

In [23]:
#one hot encoding network_output
from keras.utils import np_utils
network_output = np_utils.to_categorical(network_output)
network_output

Using TensorFlow backend.


array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [24]:
print(network_output.shape)
print(normalised_network_input.shape)

(60398, 359)
(60398, 100, 1)


In [25]:
normalised_network_input.shape[1]


100

##Model Building(LSTM)

In [0]:
from keras.layers import *
from keras.models import Sequential

In [0]:
model = Sequential()
model.add(LSTM(512, input_shape = (normalised_network_input.shape[1],normalised_network_input.shape[2]),return_sequences = True))
model.add(Dropout(0.25))
model.add(LSTM(512,return_sequences=True))
model.add(Dropout(0.25))
model.add(LSTM(256))
model.add(Dropout(0.25))
model.add(Dense(n_vocab,activation='softmax'))



In [0]:
model.compile(loss='categorical_crossentropy' , optimizer='adam' , metrics = ['accuracy'])

In [29]:
epochs_hist = model.fit(normalised_network_input, network_output, batch_size=64, epochs=10, validation_split=0.2)

Train on 48318 samples, validate on 12080 samples
Epoch 1/10
48318/48318 [==============================] - 589s 12ms/step - loss: 4.7749 - accuracy: 0.0218 - val_loss: 4.8284 - val_accuracy: 0.0328
Epoch 2/10
48318/48318 [==============================] - 594s 12ms/step - loss: 4.7341 - accuracy: 0.0239 - val_loss: 4.8355 - val_accuracy: 0.0233
Epoch 3/10
48318/48318 [==============================] - 586s 12ms/step - loss: 4.7288 - accuracy: 0.0244 - val_loss: 4.8316 - val_accuracy: 0.0171
Epoch 4/10
48318/48318 [==============================] - 586s 12ms/step - loss: 4.6917 - accuracy: 0.0249 - val_loss: 4.7322 - val_accuracy: 0.0185
Epoch 5/10
48318/48318 [==============================] - 576s 12ms/step - loss: 4.5660 - accuracy: 0.0332 - val_loss: 4.7104 - val_accuracy: 0.0213
Epoch 6/10
48318/48318 [==============================] - 576s 12ms/step - loss: 4.5026 - accuracy: 0.0399 - val_loss: 4.7386 - val_accuracy: 0.0159
Epoch 7/10
48318/48318 [==============================] 